In [22]:
import pandas as pd
import configparser


In [23]:
config = configparser.RawConfigParser()
config.read('/Users/zachblumenfeld/devtools/aura-freight-demo.ini')
HOST = config['NEO4J']['HOST']
USERNAME = config['NEO4J']['USERNAME']
PASSWORD = config['NEO4J']['PASSWORD']

In [24]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=True)

In [25]:
df = pd.read_csv('data/c2k_data_comma.csv', dtype=str)

In [26]:
[i for i in df.columns.tolist() if 'dlv' in i]

['i1_dlv_p',
 'i1_dlv_e',
 'i2_dlv_p',
 'i2_dlv_e',
 'i3_dlv_p',
 'i3_dlv_e',
 'o_dlv_p',
 'o_dlv_e']

In [27]:
#gds.run_cypher('CREATE CONSTRAINT airport_unique IF NOT EXISTS ON (n:Airport) ASSERT n.airportId  IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT airport_entry_unique IF NOT EXISTS ON (n:Entry) ASSERT n.airportId  IS UNIQUE')
#gds.run_cypher('CREATE INDEX check_in_shipment_id FOR (n:Entry)-[r:CHECK_IN]-(m:Checkpoint) IF NOT EXISTS ON (r.shipmentId)')
gds.run_cypher('CREATE INDEX check_in_shipment_id IF NOT EXISTS FOR ()-[r:CHECK_IN]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_checkpoint_unique IF NOT EXISTS ON (n:DepartureCheckpoint) ASSERT n.airportId  IS UNIQUE')
#gds.run_cypher('CREATE INDEX confirm_shipment_id FOR (:Checkpoint)-[r:CONFIRM]-(:Departure) IF NOT EXISTS ON (r.shipmentId)')
gds.run_cypher('CREATE INDEX confirm_shipment_id IF NOT EXISTS FOR ()-[r:CONFIRM]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_departure_unique IF NOT EXISTS ON (n:Departure) ASSERT n.airportId  IS UNIQUE')
#gds.run_cypher('CREATE INDEX depart_shipment_id FOR (:Departure)-[r:DEPART]-(:Arrival) IF NOT EXISTS ON (r.shipmentId)')
gds.run_cypher('CREATE INDEX depart_shipment_id IF NOT EXISTS FOR ()-[r:DEPART]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_arrival_unique IF NOT EXISTS ON (n:Arrival) ASSERT n.airportId  IS UNIQUE')
#gds.run_cypher('CREATE INDEX accept_shipment_id FOR (:Arrival)-[r:ACCEPT]-(:Checkpoint) IF NOT EXISTS ON (r.shipmentId)')
#gds.run_cypher('CREATE INDEX deliver_shipment_id FOR (:Checkpoint)-[r:DELIVER]-(:Destination) IF NOT EXISTS ON (r.shipmentId)')
gds.run_cypher('CREATE INDEX accept_shipment_id IF NOT EXISTS FOR ()-[r:ACCEPT]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_arrival_checkpoint_unique IF NOT EXISTS ON (n:ArrivalCheckpoint) ASSERT n.airportId  IS UNIQUE')
gds.run_cypher('CREATE INDEX deliver_shipment_id IF NOT EXISTS FOR ()-[r:DELIVER]-() ON (r.shipmentId)')
gds.run_cypher('CREATE CONSTRAINT airport_destination_unique  IF NOT EXISTS ON (n:Destination) ASSERT n.airportId  IS UNIQUE')
#gds.run_cypher('CREATE INDEX transfer_shipment_id FOR (:Destination)-[r:TRANSFER]-(:Entry) IF NOT EXISTS ON (r.shipmentId)')
gds.run_cypher('CREATE INDEX transfer_shipment_id IF NOT EXISTS FOR ()-[r:TRANSFER]-() ON (r.shipmentId)')
#gds.run_cypher('CREATE INDEX connect_shipment_id IF NOT EXISTS FOR (ArrivalCheckpoint)-[r:CONNECT-(DepartureCheckpoint) ON (r.shipmentId')
gds.run_cypher('CREATE INDEX connect_shipment_id IF NOT EXISTS FOR ()-[r:CONNECT]-() ON (r.shipmentId)')

""


In [28]:
# Load Nodes
cols = df.columns.tolist()

airport_ids = set()
for col in cols:
    if 'place' in col:
        #print([i for i in df.loc[(df[col] != '?') & (df[col].notna()), col].tolist()])
        airport_ids.update([int(i) for i in df.loc[(df[col] != '?') & (df[col].notna()), col].unique().tolist()])

gds.run_cypher('''
    WITH $airportIds AS airportIds
    UNWIND airportIds AS airportId
    MERGE(n0:Entry {airportId: airportId})
    MERGE(n1:DepartureCheckpoint {airportId: airportId})
    MERGE(n2:Departure {airportId: airportId})
    MERGE(n3:Arrival {airportId: airportId})
    MERGE(n4:ArrivalCheckpoint {airportId: airportId})
    MERGE(n5:Destination {airportId: airportId})
    RETURN count(n0), count(n1), count(n2), count(n3), count(n4), count(n5)
    ''', params={'airportIds':list(airport_ids)})

,count(n0),count(n1),count(n2),count(n3),count(n4),count(n5)
0,237,237,237,237,237,237


In [29]:
LEGS = [1,2,3]
SEGMENTS = [1,2,3]
GOINGS = ['i','o']
SERVICES = ['rcs', 'dlv']

In [30]:
# (n:Entry)-[r:CHECK_IN]-(m:Checkpoint) RCS inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_rcs_p', f'i{l}_rcs_e', f'i{l}_dep_1_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:Entry {{airportId: relMap["i{l}_dep_1_place"]}})
        MATCH(n2:DepartureCheckpoint {{airportId: relMap["i{l}_dep_1_place"]}})
        MERGE(n1)-[r:CHECK_IN {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber}}]->(n2)
        SET r.plannedMinutes=relMap["i{l}_rcs_p"]
        SET r.effectiveMinutes=relMap["i{l}_rcs_e"]
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [31]:
# (n:Entry)-[r:CHECK_IN]-(m:Checkpoint) RCS outbound

sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', f'o_legid', f'o_rcs_p', f'o_rcs_e', f'o_dep_1_place']].astype(int).to_dict('records')
gds.run_cypher('''
    WITH $relMaps AS relMaps
    UNWIND relMaps AS relMap
    MATCH(n1:Entry {airportId: relMap["o_dep_1_place"]})
    MATCH(n2:DepartureCheckpoint {airportId: relMap["o_dep_1_place"]})
    MERGE(n1)-[r:CHECK_IN {shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: -1}]->(n2)
    SET r.plannedMinutes=relMap["0_rcs_p"]
    SET r.effectiveMinutes=relMap["0_rcs_e"]
    RETURN count(r)
''', params={'relMaps':sub_dict})

,count(r)
0,3942


In [32]:
# (:Checkpoint)-[r:CONFIRM]-(:Departure) DEP1 inbound
# (:Departure)-[r:DEPART]-(:Arrival) DEP2 inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_dep_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
            ['nr', f'i{l}_legid', f'i{l}_dep_{s}_p', f'i{l}_dep_{s}_e', f'i{l}_dep_{s}_place', f'i{l}_rcf_{s}_place']]\
            .astype(int).to_dict('records')
        res1 = gds.run_cypher(f'''
            WITH $relMaps AS relMaps
            UNWIND relMaps AS relMap
            MATCH(n1:DepartureCheckpoint {{airportId: relMap["i{l}_dep_{s}_place"]}})
            MATCH(n2:Departure {{airportId: relMap["i{l}_dep_{s}_place"]}})
            MERGE(n1)-[r:CONFIRM {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            SET r.plannedMinutes=0
            SET r.effectiveMinutes=0
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res1)
        res2 = gds.run_cypher(f'''
            WITH $relMaps AS relMaps
            UNWIND relMaps AS relMap
            MATCH(n1:Departure {{airportId: relMap["i{l}_dep_{s}_place"]}})
            MATCH(n2:Arrival {{airportId: relMap["i{l}_rcf_{s}_place"]}})
            MERGE(n1)-[r:DEPART {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            SET r.plannedMinutes=relMap["i{l}_dep_{s}_p"]
            SET r.effectiveMinutes=relMap["i{l}_dep_{s}_e"]
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res2)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8
   count(r)
0         8


In [33]:
# (:Checkpoint)-[r:CONFIRM]-(:Departure) DEP1 outbound
# (:Departure)-[r:DEPART]-(:Arrival) DEP2 outbound
for s in SEGMENTS:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_dep_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_dep_{s}_p', f'o_dep_{s}_e', f'o_dep_{s}_place', f'o_rcf_{s}_place']]\
        .astype(int).to_dict('records')
    res1 = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:DepartureCheckpoint {{airportId: relMap["o_dep_{s}_place"]}})
        MATCH(n2:Departure {{airportId: relMap["o_dep_{s}_place"]}})
        MERGE(n1)-[r:CONFIRM {{shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        SET r.plannedMinutes=0
        SET r.effectiveMinutes=0
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res1)
    res2 = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:Departure {{airportId: relMap["o_dep_{s}_place"]}})
        MATCH(n2:Arrival {{airportId: relMap["o_rcf_{s}_place"]}})
        MERGE(n1)-[r:DEPART {{shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        SET r.plannedMinutes=relMap["o_dep_{s}_p"]
        SET r.effectiveMinutes=relMap["o_dep_{s}_e"]
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res2)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26
   count(r)
0        26


In [34]:
# (:Arrival)-[r:ACCEPT]-(:Checkpoint) inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in SEGMENTS:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_rcf_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
                          ['nr', f'i{l}_legid', f'i{l}_rcf_{s}_p', f'i{l}_rcf_{s}_e', f'i{l}_rcf_{s}_place']].astype(int).to_dict('records')
        res = gds.run_cypher(f'''
            WITH $relMaps AS relMaps
            UNWIND relMaps AS relMap
            MATCH(n1:Arrival {{airportId: relMap["i{l}_rcf_{s}_place"]}})
            MATCH(n2:ArrivalCheckpoint {{airportId: relMap["i{l}_rcf_{s}_place"]}})
            MERGE(n1)-[r:ACCEPT {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
            SET r.plannedMinutes=relMap["i{l}_rcf_{s}_p"]
            SET r.effectiveMinutes=relMap["i{l}_rcf_{s}_e"]
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l, 'segmentNumber':s})
        print(res)

== LEG 1 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      2624
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 1 ----------------------
   count(r)
0      1366
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [35]:
# (:Arrival)-[r:ACCEPT]-(:Checkpoint) RCF outbound
for s in SEGMENTS:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_rcf_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_rcf_{s}_p', f'o_rcf_{s}_e', f'o_rcf_{s}_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:Arrival {{airportId: relMap["o_rcf_{s}_place"]}})
        MATCH(n2:ArrivalCheckpoint {{airportId: relMap["o_rcf_{s}_place"]}})
        MERGE(n1)-[r:ACCEPT {{shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: $legNumber, segmentNumber: $segmentNumber}}]->(n2)
        SET r.plannedMinutes=relMap["o_rcf_{s}_p"]
        SET r.effectiveMinutes=relMap["o_rcf_{s}_e"]
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1, 'segmentNumber':s})
    print(res)

-- SEGMENT 1 ----------------------
   count(r)
0      3942
-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [36]:
# assign last inbound & outbound rcf place
def get_last_i1_rcf_place(row):
    res = row[f'i1_rcf_1_place']
    for s in [3,2]:
        if row[f'i1_rcf_{s}_place'] != '?':
            res = row[f'i1_rcf_{s}_place']
    return res
df['last_i1_rcf_place'] = df.apply(get_last_i1_rcf_place, axis =1)

def get_last_i2_rcf_place(row):
    res = row[f'i2_rcf_1_place']
    for s in [3,2]:
        if row[f'i2_rcf_{s}_place'] != '?':
            res = row[f'i2_rcf_{s}_place']
    return res
df['last_i2_rcf_place'] = df.apply(get_last_i2_rcf_place, axis =1)

def get_last_i3_rcf_place(row):
    res = row[f'i3_rcf_1_place']
    for s in [3,2]:
        if row[f'i3_rcf_{s}_place'] != '?':
            res = row[f'i3_rcf_{s}_place']
    return res
df['last_i3_rcf_place'] = df.apply(get_last_i3_rcf_place, axis =1)

def get_last_outbound_rcf_place(row):
    for s in [3,2,1]:
        if row[f'o_rcf_{s}_place'] != '?':
            return row[f'o_rcf_{s}_place']
    raise Exception("cannot find last rcf place")
df['last_o_rcf_place'] = df.apply(get_last_outbound_rcf_place, axis =1)

df

,nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,...,o_rcf_3_e,o_rcf_3_place,o_dlv_p,o_dlv_e,o_hops,legs,last_i1_rcf_place,last_i2_rcf_place,last_i3_rcf_place,last_o_rcf_place
0,0,5182,199,218,210,215,609,935,736,256,...,?,?,780,434,1,2,256,256,?,411
1,1,6523,844,584,90,297,700,1935,1415,431,...,?,?,3870,445,1,2,431,431,?,256
2,2,5878,4380,4119,90,280,456,905,547,700,...,?,?,550,1520,1,1,700,?,?,349
3,3,1275,759,169,240,777,173,340,577,349,...,?,?,3780,159,1,1,671,?,?,700
4,4,8117,1597,1485,150,241,411,585,612,128,...,?,?,4140,4797,2,1,166,?,?,411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,3939,4225,122,19,240,278,815,830,761,174,...,?,?,1665,1300,2,3,174,174,174,293
3939,3940,14017,2028,413,270,1825,605,2610,2535,349,...,?,?,3780,807,2,1,597,?,?,113
3940,3941,4660,1356,178,240,1359,815,760,716,609,...,?,?,5100,4381,2,1,609,?,?,737
3941,3942,6472,2692,1856,90,867,700,1060,1049,113,...,?,?,3780,945,2,2,113,113,?,635


In [37]:
# (:Checkpoint)-[r:DELIVER]-(:Destination) DLV inbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', f'i{l}_dlv_p', f'i{l}_dlv_e', 'last_i1_rcf_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:ArrivalCheckpoint {{airportId: relMap["last_i1_rcf_place"]}})
        MATCH(n2:Destination {{airportId: relMap["last_i1_rcf_place"]}})
        MERGE(n1)-[r:DELIVER {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber}}]->(n2)
        SET r.plannedMinutes=relMap["i{l}_dlv_p"]
        SET r.effectiveMinutes=relMap["i{l}_dlv_e"]
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [38]:
# (:Checkpoint)-[r:DELIVER]-(:Destination) DLV outbound
sub_dict = df.loc[(df[f'o_legid'] != '?') & (df[f'o_legid'].notna()),
                  ['nr', 'o_legid', f'o_dlv_p', f'o_dlv_e', 'last_o_rcf_place']].astype(int).to_dict('records')
res = gds.run_cypher('''
    WITH $relMaps AS relMaps
    UNWIND relMaps AS relMap
    MATCH(n1:ArrivalCheckpoint {airportId: relMap["last_o_rcf_place"]})
    MATCH(n2:Destination {airportId: relMap["last_o_rcf_place"]})
    MERGE(n1)-[r:DELIVER {shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: $legNumber}]->(n2)
    SET r.plannedMinutes=relMap["o_dlv_p"]
    SET r.effectiveMinutes=relMap["o_dlv_e"]
    RETURN count(r)
''', params={'relMaps':sub_dict, 'legNumber':-1})
print(res)

   count(r)
0      3942


In [39]:
# (:Destination)-[r:TRANSFER]-(:Entry) inbound->outbound
for l in LEGS:
    sub_dict = df.loc[(df[f'i{l}_legid'] != '?') & (df[f'i{l}_legid'].notna()),
                      ['nr', f'i{l}_legid', 'o_legid', 'last_i1_rcf_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:Destination {{airportId: relMap["last_i1_rcf_place"]}})
        MATCH(n2:Entry {{airportId: relMap["last_i1_rcf_place"]}})
        MERGE(n1)-[r:TRANSFER {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber}}]->(n2)
        SET r.plannedMinutes=0
        SET r.effectiveMinutes=0
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':l})
    print(res)

   count(r)
0      3942
   count(r)
0      2624
   count(r)
0      1366


In [40]:
# (:ArrivalCheckpoint)-[r:CONNECT]-(:DepartureCheckpoint) inbound
for l in LEGS:
    print(f'== LEG {l} ======================')
    for s in [2,3]:
        print(f'-- SEGMENT {s} ----------------------')
        sub_dict = df.loc[(df[f'i{l}_rcf_{s}_place'] != '?') & (df[f'i{l}_legid'].notna()),
                          ['nr', f'i{l}_legid', f'i{l}_dep_{s}_place']].astype(int).to_dict('records')
        res = gds.run_cypher(f'''
            WITH $relMaps AS relMaps
            UNWIND relMaps AS relMap
            MATCH(n1:ArrivalCheckpoint {{airportId: relMap["i{l}_dep_{s}_place"]}})
            MATCH(n2:DepartureCheckpoint {{airportId: relMap["i{l}_dep_{s}_place"]}})
            MERGE(n1)-[r:CONNECT {{shipmentId: relMap["nr"], legId: relMap["i{l}_legid"], legNumber: $legNumber}}]->(n2)
            SET r.plannedMinutes=0
            SET r.effectiveMinutes=0
            RETURN count(r)
        ''', params={'relMaps':sub_dict, 'legNumber':l})
        print(res)

== LEG 1 ======================
-- SEGMENT 2 ----------------------
   count(r)
0      1195
-- SEGMENT 3 ----------------------
   count(r)
0        23
== LEG 2 ======================
-- SEGMENT 2 ----------------------
   count(r)
0       791
-- SEGMENT 3 ----------------------
   count(r)
0        14
== LEG 3 ======================
-- SEGMENT 2 ----------------------
   count(r)
0       391
-- SEGMENT 3 ----------------------
   count(r)
0         8


In [41]:
# (:ArrivalCheckpoint)-[r:CONNECT]-(:DepartureCheckpoint) outbound

for s in [2,3]:
    print(f'-- SEGMENT {s} ----------------------')
    sub_dict = df.loc[(df[f'o_rcf_{s}_place'] != '?') & (df[f'o_legid'].notna()),
                      ['nr', f'o_legid', f'o_dep_{s}_place']].astype(int).to_dict('records')
    res = gds.run_cypher(f'''
        WITH $relMaps AS relMaps
        UNWIND relMaps AS relMap
        MATCH(n1:ArrivalCheckpoint {{airportId: relMap["o_dep_{s}_place"]}})
        MATCH(n2:DepartureCheckpoint {{airportId: relMap["o_dep_{s}_place"]}})
        MERGE(n1)-[r:CONNECT {{shipmentId: relMap["nr"], legId: relMap["o_legid"], legNumber: $legNumber}}]->(n2)
        SET r.plannedMinutes=0
        SET r.effectiveMinutes=0
        RETURN count(r)
    ''', params={'relMaps':sub_dict, 'legNumber':-1})
    print(res)

-- SEGMENT 2 ----------------------
   count(r)
0      1845
-- SEGMENT 3 ----------------------
   count(r)
0        26


In [21]:
gds.run_cypher('MATCH(n) DETACH DELETE n')
gds.run_cypher('CALL apoc.schema.assert({},{})')

,label,key,keys,unique,action
0,CHECK_IN,shipmentId,[shipmentId],False,DROPPED
1,CONFIRM,shipmentId,[shipmentId],False,DROPPED
2,DEPART,shipmentId,[shipmentId],False,DROPPED
3,ACCEPT,shipmentId,[shipmentId],False,DROPPED
4,DELIVER,shipmentId,[shipmentId],False,DROPPED
5,TRANSFER,shipmentId,[shipmentId],False,DROPPED
6,CONNECT,shipmentId,[shipmentId],False,DROPPED
7,Entry,airportId,[airportId],True,DROPPED
8,DepartureCheckpoint,airportId,[airportId],True,DROPPED
9,Departure,airportId,[airportId],True,DROPPED
